//Iterated, own window and mirrored\\

In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw hand landmarks on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

    # Display the image in its own window
    cv2.imshow('Hand Tracking', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\dsrus\Desktop\Workspace\MTLiens\pytorch_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


//Black on White Chunks\\

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

fig, ax = plt.subplots(figsize=(10, 10))

GRID_SIZE = 50
UPDATE_FREQUENCY = 2  # Update display every 2 frames

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Create a GRID_SIZE x GRID_SIZE grid of 'O's
    grid = np.full((GRID_SIZE, GRID_SIZE), 'O', dtype='<U1')

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # Convert landmark coordinates to grid coordinates
                x = int(landmark.x * GRID_SIZE)
                y = int(landmark.y * GRID_SIZE)
                
                # Mark a 3x3 area around each landmark
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                            grid[ny, nx] = 'X'

    frame_count += 1
    if frame_count % UPDATE_FREQUENCY == 0:
        # Display the grid
        ax.clear()
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                color = 'black' if grid[i, j] == 'X' else 'white'
                ax.text(j, i, grid[i, j], ha='center', va='center', color=color, fontweight='bold')
        ax.set_xlim(-1, GRID_SIZE)
        ax.set_ylim(GRID_SIZE, -1)
        ax.axis('off')
        
        clear_output(wait=True)
        display(fig)
        plt.pause(0.01)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
hands.close()
plt.close(fig)

# Ensure the camera is released
cv2.destroyAllWindows()

//4elements Game\\

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from playsound import playsound
import threading

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            playsound(sound_file)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# To keep track of which sounds have been played
sound_played = {element: False for element in squares}

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)
        cv2.putText(image, element, (info['position'][0] + 10, info['position'][1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            # Check if hand is touching any square
            for element, info in squares.items():
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)
                if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                    print(f"Touch detected in {element} square")
                    if not sound_played[element]:
                        print(f"Attempting to play sound for {element}")
                        play_sound(info['sound'])
                        sound_played[element] = True
                else:
                    sound_played[element] = False
        

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

//Using Winsound\\

In [15]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played
sound_played = {element: False for element in squares}

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)
        cv2.putText(image, element, (info['position'][0] + 10, info['position'][1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            # Check if hand is touching any square
            for element, info in squares.items():
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)
                if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                    print(f"Touch detected in {element} square")
                    if not sound_played[element]:
                        print(f"Attempting to play sound for {element}")
                        play_sound(info['sound'])
                        sound_played[element] = True
                else:
                    sound_played[element] = False

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

Touch detected in Earth square
Attempting to play sound for Earth
Touch detected in Earth square
Attempting to play sound for Earth
Touch detected in Air square
Attempting to play sound for Air
Touch detected in Air square
Attempting to play sound for Air
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav


In [ ]:
//Only 1 time sound, better interation\\

In [18]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire5.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)
        cv2.putText(image, element, (info['position'][0] + 10, info['position'][1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            # Check if hand is touching any square
            for element, info in squares.items():
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)
                if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                    if not finger_in_box[element]:
                        print(f"Touch detected in {element} square")
                        finger_in_box[element] = True
                        if not sound_played[element]:
                            print(f"Attempting to play sound for {element}")
                            play_sound(info['sound'])
                            sound_played[element] = True
                else:
                    finger_in_box[element] = False
                    sound_played[element] = False

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

Touch detected in Air square
Attempting to play sound for Air
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav


//Word Grab\\

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}

# Word grabbing and moving
grabbed_word = None
word_position = None

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)
        if element != grabbed_word:
            cv2.putText(image, element, (info['position'][0] + 10, info['position'][1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_position = (x, y)
                else:
                    grabbed_word = None
                    word_position = None
            else:
                for element, info in squares.items():
                    if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if not sound_played[element]:
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                sound_played[element] = True
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                            word_position = (x, y)
                    else:
                        finger_in_box[element] = False
                        sound_played[element] = False

    # Draw grabbed word
    if grabbed_word and word_position:
        cv2.putText(image, grabbed_word, word_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

c:\Users\dsrus\Desktop\Workspace\MTLiens\pytorch_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Touch detected in Air square
Attempting to play sound for Air
Touch detected in Air square
Attempting to play sound for Air
Touch detected in Air square
Attempting to play sound for Air
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Touch detected in Water square
Attempting to play sound for Water
Touch detected in Water square
Attempting to play sound for Water
Touch detected in Water square
Attempting to play sound for Water
Touch detected in Water square
Attempting to play sound for Water
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav


//Word drop correct placement\\

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None
word_positions = {element: info['position'] for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Draw words at their current positions
    for element, position in word_positions.items():
        cv2.putText(image, element, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

//Better center words\\

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Function to get centered text position
def get_centered_text_position(text, box_position, box_size):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
    text_x = box_position[0] + (box_size - text_size[0]) // 2
    text_y = box_position[1] + (box_size + text_size[1]) // 2
    return (text_x, text_y)

# Initialize word positions at the center of each box
word_positions = {element: get_centered_text_position(element, info['position'], square_size) for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if info['position'][0] < x < info['position'][0] + square_size and info['position'][1] < y < info['position'][1] + square_size:
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Draw words at their current positions
    for element, position in word_positions.items():
        cv2.putText(image, element, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

//Gold creation script\\

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Define gold box
gold_box = {'color': (0, 215, 255), 'position': ((width - square_size) // 2, height - square_size)}

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Function to get centered text position
def get_centered_text_position(text, box_position, box_size):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
    text_x = box_position[0] + (box_size - text_size[0]) // 2
    text_y = box_position[1] + (box_size + text_size[1]) // 2
    return (text_x, text_y)

# Initialize word positions at the center of each box
word_positions = {element: get_centered_text_position(element, info['position'], square_size) for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

def is_point_in_box(point, box_position, box_size):
    return (box_position[0] < point[0] < box_position[0] + box_size and
            box_position[1] < point[1] < box_position[1] + box_size)

gold_achieved = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw gold box
    overlay = image.copy()
    cv2.rectangle(overlay, gold_box['position'], (gold_box['position'][0] + square_size, gold_box['position'][1] + square_size), gold_box['color'], -1)
    cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if is_point_in_box((x, y), info['position'], square_size):
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Check if all words are in the gold box
    if not gold_achieved:
        words_in_gold = sum(1 for word, pos in word_positions.items() if is_point_in_box(pos, gold_box['position'], square_size))
        if words_in_gold == 4:
            gold_achieved = True
            word_positions.clear()  # Erase all words

    # Draw words at their current positions or "Gold" if achieved
    if gold_achieved:
        gold_text_pos = get_centered_text_position("Gold", gold_box['position'], square_size)
        cv2.putText(image, "Gold", gold_text_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    else:
        for element, position in word_positions.items():
            cv2.putText(image, element, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

In [ ]:
//Eureka\\

In [43]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    'Fire': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    'Air': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    'Water': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    'Earth': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Define gold box
gold_box = {'color': (0, 215, 255), 'position': ((width - square_size) // 2, height - square_size)}

# Eureka sound
eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

if not os.path.exists(eureka_sound):
    print(f"Warning: Eureka sound file not found: {eureka_sound}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Function to get centered text position
def get_centered_text_position(text, box_position, box_size):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
    text_x = box_position[0] + (box_size - text_size[0]) // 2
    text_y = box_position[1] + (box_size + text_size[1]) // 2
    return (text_x, text_y)

# Initialize word positions at the center of each box
word_positions = {element: get_centered_text_position(element, info['position'], square_size) for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

def is_point_in_box(point, box_position, box_size):
    return (box_position[0] < point[0] < box_position[0] + box_size and
            box_position[1] < point[1] < box_position[1] + box_size)

gold_achieved = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw gold box
    overlay = image.copy()
    cv2.rectangle(overlay, gold_box['position'], (gold_box['position'][0] + square_size, gold_box['position'][1] + square_size), gold_box['color'], -1)
    cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if is_point_in_box((x, y), info['position'], square_size):
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Check if all words are in the gold box
    if not gold_achieved:
        words_in_gold = sum(1 for word, pos in word_positions.items() if is_point_in_box(pos, gold_box['position'], square_size))
        if words_in_gold == 4:
            gold_achieved = True
            word_positions.clear()  # Erase all words
            play_sound(eureka_sound)  # Play the Eureka sound

    # Draw words at their current positions or "Gold" if achieved
    if gold_achieved:
        gold_text_pos = get_centered_text_position("Gold", gold_box['position'], square_size)
        cv2.putText(image, "Gold", gold_text_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    else:
        for element, position in word_positions.items():
            cv2.putText(image, element, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

Touch detected in Fire square
Attempting to play sound for Fire
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav


//Emoji's, but weird color thing\\

In [50]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    '🔥': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    '💨': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    '🌊': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    '🌱': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Define gold box
gold_box = {'color': (0, 215, 255), 'position': ((width - square_size) // 2, height - square_size)}

# Eureka sound
eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

if not os.path.exists(eureka_sound):
    print(f"Warning: Eureka sound file not found: {eureka_sound}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Initialize word positions at the center of each box
word_positions = {element: (info['position'][0] + square_size // 2, info['position'][1] + square_size // 2) for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

def is_point_in_box(point, box_position, box_size):
    return (box_position[0] < point[0] < box_position[0] + box_size and
            box_position[1] < point[1] < box_position[1] + box_size)

gold_achieved = False

# Load a font that supports emoji
font = ImageFont.truetype("seguiemj.ttf", 40)  # You may need to specify the full path to the font file

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)

    # Convert back to BGR for OpenCV display
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw squares and labels
    for element, info in squares.items():
        overlay = image.copy()
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Draw gold box
    overlay = image.copy()
    cv2.rectangle(overlay, gold_box['position'], (gold_box['position'][0] + square_size, gold_box['position'][1] + square_size), gold_box['color'], -1)
    cv2.addWeighted(overlay, 0.25, image, 0.75, 0, image)

    # Convert the image to PIL Image for drawing emoji
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if is_point_in_box((x, y), info['position'], square_size):
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Check if all words are in the gold box
    if not gold_achieved:
        words_in_gold = sum(1 for word, pos in word_positions.items() if is_point_in_box(pos, gold_box['position'], square_size))
        if words_in_gold == 4:
            gold_achieved = True
            word_positions.clear()  # Erase all words
            play_sound(eureka_sound)  # Play the Eureka sound

    # Draw words at their current positions or "Gold" if achieved
    if gold_achieved:
        draw.text((gold_box['position'][0] + square_size // 2, gold_box['position'][1] + square_size // 2), "Gold", font=font, fill=(255, 255, 255), anchor="mm")
    else:
        for element, position in word_positions.items():
            draw.text(position, element, font=font, fill=(255, 255, 255), anchor="mm")

    # Convert PIL Image back to OpenCV image
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

    # Display the image in its own window
    cv2.imshow('Element Squares', image)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

//Emoji Font, fixed?\\

In [24]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont
import emoji

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    '🔥': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    '💨': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    '🌊': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    '🌱': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Define gold box
gold_box = {'color': (0, 215, 255), 'position': ((width - square_size) // 2, height - square_size)}

# Eureka sound
eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

if not os.path.exists(eureka_sound):
    print(f"Warning: Eureka sound file not found: {eureka_sound}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Initialize word positions at the center of each box
word_positions = {element: (info['position'][0] + square_size // 2, info['position'][1] + square_size // 2) for element, info in squares.items()}

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

def is_point_in_box(point, box_position, box_size):
    return (box_position[0] < point[0] < box_position[0] + box_size and
            box_position[1] < point[1] < box_position[1] + box_size)

gold_achieved = False

# Prepare emoji images
emoji_images = {}
for element in squares.keys():
    emoji_img = emoji.emojize(element, language='alias')
    emoji_images[element] = Image.new('RGBA', (64, 64), (255, 255, 255, 0))
    d = ImageDraw.Draw(emoji_images[element])
    d.text((32, 32), emoji_img, font=ImageFont.truetype("seguiemj.ttf", 48), fill=(0, 0, 0, 255), anchor="mm")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image_rgb)

    # Draw squares and labels
    overlay = image_rgb.copy()
    for element, info in squares.items():
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
    cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

    # Draw gold box
    cv2.rectangle(overlay, gold_box['position'], (gold_box['position'][0] + square_size, gold_box['position'][1] + square_size), gold_box['color'], -1)
    cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

    # Convert the image to PIL Image for drawing emoji
    pil_image = Image.fromarray(image_rgb)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if is_point_in_box((x, y), info['position'], square_size):
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Check if all words are in the gold box
    if not gold_achieved:
        words_in_gold = sum(1 for word, pos in word_positions.items() if is_point_in_box(pos, gold_box['position'], square_size))
        if words_in_gold == 4:
            gold_achieved = True
            word_positions.clear()  # Erase all words
            play_sound(eureka_sound)  # Play the Eureka sound

    # Draw words at their current positions or "Gold" if achieved
    if gold_achieved:
        draw = ImageDraw.Draw(pil_image)
        draw.text((gold_box['position'][0] + square_size // 2, gold_box['position'][1] + square_size // 2), "Gold", font=ImageFont.truetype("arial.ttf", 40), fill=(255, 255, 255), anchor="mm")
    else:
        for element, position in word_positions.items():
            pil_image.paste(emoji_images[element], (int(position[0] - 32), int(position[1] - 32)), emoji_images[element])

    # Convert PIL Image back to OpenCV image (RGB)
    image_rgb = np.array(pil_image)

    # Convert RGB to BGR for display
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Display the image in its own window
    cv2.imshow('Element Squares', image_bgr)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

Touch detected in 💨 square
Attempting to play sound for 💨
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav


//Pretty good, with 'r' reset\\ 

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont
import emoji

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Function to play sound asynchronously
def play_sound(sound_file):
    def play():
        try:
            if not os.path.exists(sound_file):
                print(f"Error: Sound file not found: {sound_file}")
                return
            winsound.PlaySound(sound_file, winsound.SND_FILENAME)
            print(f"Played sound: {sound_file}")
        except Exception as e:
            print(f"Error playing sound {sound_file}: {e}")

    threading.Thread(target=play, daemon=True).start()

cap = cv2.VideoCapture(0)

# Get frame dimensions
ret, frame = cap.read()
height, width = frame.shape[:2]

# Define squares
square_size = min(width, height) // 4
squares = {
    ':fire:': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
    ':dash:': {'color': (255, 255, 0), 'position': (width - square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
    ':ocean:': {'color': (255, 0, 0), 'position': (0, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
    ':seedling:': {'color': (0, 128, 128), 'position': (width - square_size, height - square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
}

# Define gold box
gold_box = {'color': (0, 215, 255), 'position': ((width - square_size) // 2, height - square_size)}

# Eureka sound
eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'

# Check if sound files exist
for element, info in squares.items():
    if not os.path.exists(info['sound']):
        print(f"Warning: Sound file not found for {element}: {info['sound']}")

if not os.path.exists(eureka_sound):
    print(f"Warning: Eureka sound file not found: {eureka_sound}")

# To keep track of which sounds have been played and finger position
sound_played = {element: False for element in squares}
finger_in_box = {element: False for element in squares}
last_sound_time = {element: 0 for element in squares}

# Word grabbing and moving
grabbed_word = None

# Initialize word positions at the center of each box
def reset_word_positions():
    global word_positions, gold_achieved, grabbed_word
    word_positions = {element: (info['position'][0] + square_size // 2, info['position'][1] + square_size // 2) for element, info in squares.items()}
    gold_achieved = False
    grabbed_word = None

reset_word_positions()

def is_hand_closed(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

def is_point_in_box(point, box_position, box_size):
    return (box_position[0] < point[0] < box_position[0] + box_size and
            box_position[1] < point[1] < box_position[1] + box_size)

# Prepare emoji images
emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
emoji_images = {}
for element in squares.keys():
    emoji_char = emoji.emojize(element, language='alias')
    emoji_images[element] = Image.new('RGBA', (64, 64), (255, 255, 255, 0))
    d = ImageDraw.Draw(emoji_images[element])
    d.text((32, 32), emoji_char, font=emoji_font, fill=(0, 0, 0, 255), anchor="mm")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Mirror the frame
    frame = cv2.flip(frame, 1)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image_rgb)

    # Draw squares and labels
    overlay = image_rgb.copy()
    for element, info in squares.items():
        cv2.rectangle(overlay, info['position'], (info['position'][0] + square_size, info['position'][1] + square_size), info['color'], -1)
    cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

    # Draw gold box
    cv2.rectangle(overlay, gold_box['position'], (gold_box['position'][0] + square_size, gold_box['position'][1] + square_size), gold_box['color'], -1)
    cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

    # Convert the image to PIL Image for drawing emoji
    pil_image = Image.fromarray(image_rgb)

    # Draw hand landmarks and check for touches
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )
            
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)

            hand_closed = is_hand_closed(hand_landmarks)

            if grabbed_word:
                if hand_closed:
                    word_positions[grabbed_word] = (x, y)
                else:
                    grabbed_word = None
            else:
                for element, info in squares.items():
                    if is_point_in_box((x, y), info['position'], square_size):
                        current_time = time.time()
                        if not finger_in_box[element]:
                            print(f"Touch detected in {element} square")
                            finger_in_box[element] = True
                            if current_time - last_sound_time[element] > 1:  # 1 second cooldown
                                print(f"Attempting to play sound for {element}")
                                play_sound(info['sound'])
                                last_sound_time[element] = current_time
                        if hand_closed and not grabbed_word:
                            grabbed_word = element
                    else:
                        finger_in_box[element] = False

    # Check if all words are in the gold box
    if not gold_achieved:
        words_in_gold = sum(1 for word, pos in word_positions.items() if is_point_in_box(pos, gold_box['position'], square_size))
        if words_in_gold == 4:
            gold_achieved = True
            word_positions.clear()  # Erase all words
            play_sound(eureka_sound)  # Play the Eureka sound

    # Draw words at their current positions or "Gold" if achieved
    if gold_achieved:
        draw = ImageDraw.Draw(pil_image)
        draw.text((gold_box['position'][0] + square_size // 2, gold_box['position'][1] + square_size // 2), "Gold", font=ImageFont.truetype("arial.ttf", 40), fill=(255, 255, 255), anchor="mm")
    else:
        for element, position in word_positions.items():
            emoji_char = emoji.emojize(element, language='alias')
            draw = ImageDraw.Draw(pil_image)
            draw.text((position[0], position[1]), emoji_char, font=emoji_font, fill=(255, 255, 255), anchor="mm")

    # Convert PIL Image back to OpenCV image (RGB)
    image_rgb = np.array(pil_image)

    # Convert RGB to BGR for display
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Display the image in its own window
    cv2.imshow('Element Squares', image_bgr)

    # Check for key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        reset_word_positions()
        print("Game reset!")

cap.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # Stop any ongoing sounds

Touch detected in :fire: square
Attempting to play sound for :fire:
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav
Touch detected in :dash: square
Attempting to play sound for :dash:
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Touch detected in :ocean: square
Attempting to play sound for :ocean:
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Touch detected in :seedling: square
Attempting to play sound for :seedling:
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav


//General MediaPipe mask test\\

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Selfie Segmentation
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=0)

# Initialize the camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the image horizontally for a later selfie-view display
    image = cv2.flip(image, 1)

    # Convert the BGR image to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and get the segmentation mask
    results = selfie_segmentation.process(rgb_image)

    # Create a blue color image
    blue_color = np.zeros(image.shape, dtype=np.uint8)
    blue_color[:] = (255, 0, 0)  # BGR format

    # Create the segmentation mask image
    segmentation_mask = results.segmentation_mask

    # Ensure the mask has 3 channels like the image
    segmentation_mask_3channel = np.stack((segmentation_mask,)*3, axis=-1)

    # Blend the original image with the blue color based on the mask
    output_image = np.where(segmentation_mask_3channel > 0.1, 
                            cv2.addWeighted(image, 0.5, blue_color, 0.5, 0), 
                            image)

    # Display the result
    cv2.imshow('Selfie Segmentation', output_image)

    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

//Final, extra emoji thing but wtv\\

In [57]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont
import emoji

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            ':fire:': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
            ':dash:': {'color': (255, 255, 0), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
            ':ocean:': {'color': (255, 0, 0), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
            ':seedling:': {'color': (0, 128, 128), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
        self.emoji_images = self.prepare_emoji_images()

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def prepare_emoji_images(self):
        emoji_images = {}
        for element in self.squares.keys():
            emoji_char = emoji.emojize(element, language='alias')
            emoji_images[element] = Image.new('RGBA', (64, 64), (255, 255, 255, 0))
            d = ImageDraw.Draw(emoji_images[element])
            d.text((32, 32), emoji_char, font=self.emoji_font, fill=(0, 0, 0, 255), anchor="mm")
        return emoji_images

    def play_sound(self, sound_file):
        def play():
            try:
                if not os.path.exists(sound_file):
                    print(f"Error: Sound file not found: {sound_file}")
                    return
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      self.gold_box['color'] if self.mask_color is None else self.mask_color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        for element, position in self.word_positions.items():
            emoji_char = emoji.emojize(element, language='alias')
            draw = ImageDraw.Draw(pil_image)
            draw.text((position[0], position[1]), emoji_char, font=self.emoji_font, fill=(255, 255, 255), anchor="mm")

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        print(f"Switched to {'game' if self.is_game_view else 'mask'} view")

    def apply_mask(self, frame, mask_color):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if mask_color is not None:
            overlay = np.zeros(image_rgb.shape, dtype=np.uint8)
            overlay[:] = mask_color
            overlay = cv2.addWeighted(image_rgb, 0.8, overlay, 0.2, 0)
            image_rgb = np.where(condition, overlay, image_rgb)
        return image_rgb

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
            else:
                image_rgb = self.camera_manager.apply_mask(frame, self.game.mask_color)

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()

Touch detected in :ocean: square
Attempting to play sound for :ocean:
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Switched to mask view
Switched to game view
Switched to mask view
Switched to game view
Switched to mask view
Switched to game view


//Mask immediate\\

In [62]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont
import emoji

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize MediaPipe Selfie Segmentation
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            ':fire:': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
            ':dash:': {'color': (255, 255, 0), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
            ':ocean:': {'color': (255, 0, 0), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
            ':seedling:': {'color': (0, 128, 128), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
        self.emoji_images = self.prepare_emoji_images()

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def prepare_emoji_images(self):
        emoji_images = {}
        for element in self.squares.keys():
            emoji_char = emoji.emojize(element, language='alias')
            emoji_images[element] = Image.new('RGBA', (64, 64), (255, 255, 255, 0))
            d = ImageDraw.Draw(emoji_images[element])
            d.text((32, 32), emoji_char, font=self.emoji_font, fill=(0, 0, 0, 255), anchor="mm")
        return emoji_images

    def play_sound(self, sound_file):
        def play():
            try:
                if not os.path.exists(sound_file):
                    print(f"Error: Sound file not found: {sound_file}")
                    return
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def apply_mask(self, image_rgb):
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if self.mask_color is not None:
            overlay = np.zeros(image_rgb.shape, dtype=np.uint8)
            overlay[:] = self.mask_color
            overlay = cv2.addWeighted(image_rgb, 0.8, overlay, 0.2, 0)
            image_rgb = np.where(condition, overlay, image_rgb)
        return image_rgb

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        # Apply mask first
        image_rgb = self.apply_mask(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      self.gold_box['color'] if self.mask_color is None else self.mask_color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        for element, position in self.word_positions.items():
            emoji_char = emoji.emojize(element, language='alias')
            draw = ImageDraw.Draw(pil_image)
            draw.text((position[0], position[1]), emoji_char, font=self.emoji_font, fill=(255, 255, 255), anchor="mm")

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        print(f"Switched to {'game' if self.is_game_view else 'mask'} view")

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
            else:
                image_rgb = self.game.apply_mask(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()

Touch detected in :fire: square
Attempting to play sound for :fire:
Switched to mask view
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav
Switched to game view
Touch detected in :ocean: square
Attempting to play sound for :ocean:
Switched to mask view
Switched to game view
Switched to mask view
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Switched to game view


//Slight manual iteration attempt\\

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize MediaPipe Selfie Segmentation
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            '🔥': {'color': (0, 0, 255), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
            '💨': {'color': (255, 255, 0), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
            '🌊': {'color': (255, 0, 0), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
            '🌱': {'color': (0, 128, 128), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def play_sound(self, sound_file):
        def play():
            try:
                if not os.path.exists(sound_file):
                    print(f"Error: Sound file not found: {sound_file}")
                    return
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def apply_mask(self, image_rgb):
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if self.mask_color is not None:
            overlay = np.zeros(image_rgb.shape, dtype=np.uint8)
            overlay[:] = self.mask_color
            overlay = cv2.addWeighted(image_rgb, 0.8, overlay, 0.2, 0)
            image_rgb = np.where(condition, overlay, image_rgb)
        return image_rgb

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        # Apply mask first
        image_rgb = self.apply_mask(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      self.gold_box['color'] if self.mask_color is None else self.mask_color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        for element, position in self.word_positions.items():
            draw = ImageDraw.Draw(pil_image)
            draw.text((position[0], position[1]), element, font=self.emoji_font, fill=(255, 255, 255), anchor="mm")

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        print(f"Switched to {'game' if self.is_game_view else 'mask'} view")

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
            else:
                image_rgb = self.game.apply_mask(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()

c:\Users\dsrus\Desktop\Workspace\MTLiens\pytorch_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Touch detected in 🔥 square
Attempting to play sound for 🔥
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav
Touch detected in 🌊 square
Attempting to play sound for 🌊
Touch detected in 🌊 square
Touch detected in 💨 square
Attempting to play sound for 💨
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav
Touch detected in 🌱 square
Attempting to play sound for 🌱
Touch detected in 🌱 square
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav
Switched to mask view
Switched to game view
Game reset!
